In [1]:
# -*- coding:utf-8 -*-
#
# Convergence of Finite Differences on a 2D Poisson problem
# Emmanuel Dormy
# Numerical Methods for Fluid Dynamics, ENS 2023,
# TD1
#

from sys import exit
import numpy as np
np.set_printoptions(threshold=np.inf, linewidth=1800)
import scipy.sparse as sp
import scipy.sparse.linalg as lg
import matplotlib.pyplot as plt
import numpy.linalg as la


def BuildLaPoisson():
    """
    Matrice du Laplacien 2D
    """

    # Definition of the 1D Lalace operator

    NXi = NX - 2
    NYi = NY - 2

    # AXE X
    # Diagonal terms
    dataNX = [np.ones(NXi), -2 * np.ones(NXi), np.ones(NXi)]

    # AXE Y
    # Diagonal terms
    dataNY = [np.ones(NYi), -2 * np.ones(NYi), np.ones(NYi)]

    # Their positions
    offsets = np.array([-1, 0, 1])
    DXX = sp.dia_matrix((dataNX, offsets), shape=(NXi, NXi)) * dx_2
    DYY = sp.dia_matrix((dataNY, offsets), shape=(NYi, NYi)) * dy_2
    # print(DXX.todense())
    # print(DYY.todense())

    # 2D Laplace operator
    LAP = sp.kron(sp.eye(NYi, NYi), DXX) + sp.kron(DYY, sp.eye(NXi, NXi))

    # print(LAP.todense())
    return LAP


def LUdecomposition(LAP):
    """
    returns the LU decomposition of a sparse matrix LAP
    """
    return lg.splu(LAP.tocsc(),)


def ResoLap(splu, RHS):
    """
    solve the system

    SPLU * x = RHS

    Args:
    --RHS: 2D array((NY,NX))
    --splu: LU decomposed matrix shape (NY*NX, NY*NX)

    Return: x = array[NY,NX]

    """
    # array 2D -> array 1D
    f2 = RHS.ravel()

    # Solving the linear system
    x = splu.solve(f2)

    return x.reshape(RHS.shape)


#########################################
# MAIN: Programme principal
#########################################


# Taille adimensionnee du domaine
# aspect_ratio = LY/LX

aspect_ratio = float(2.)
LY = float(1.)
LX = LY / aspect_ratio

# GRID RESOLUTION

# Taille des tableaux (points fantomes inclus)
NX = int(10)
NY = int(6)

# Allocate f and u
f = np.zeros((NY, NX))
u = np.zeros((NY, NX))

# Elements differentiels
dx = LX / (NX - 1)
dy = LY / (NY - 1)

dx_2 = 1. / (dx * dx)
dy_2 = 1. / (dy * dy)


# Maillage
x = np.linspace(0, LX, NX)
y = np.linspace(0, LY, NY)
[xx, yy] = np.meshgrid(x, y)


# Matrix construction
LAPoisson = BuildLaPoisson()
LUPoisson = LUdecomposition(LAPoisson)

# Expression of the RHS
f = ((np.pi / LX)**2 + (np.pi / LY)**2) * \
    np.sin(np.pi * xx / LX) * np.sin(np.pi * yy / LY)

# Solve the linear system
u[1:-1, 1:-1] = ResoLap(LUPoisson, RHS=f[1:-1, 1:-1])

# FIGURE draw works only if plt.ion()


def BuildLAPN():
    """
    Laplacian matrix for Phi with Neumann BC

    The value is set at one point (here [0][1]) to ensure uniqueness

    """
    # Dropping ghost points (-2)
    NXi = nx
    NYi = ny

    # 1D Laplace operator

    # X-axis
    # Diagonal terms
    dataNXi = [np.ones(NXi), -2 * np.ones(NXi), np.ones(NXi)]

    # Boundary conditions
    dataNXi[1][0] = -1.  # Neumann
    dataNXi[1][-1] = -1.  # Neumann

    # Y-axis
    # Diagonal terms
    dataNYi = [np.ones(NYi), -2 * np.ones(NYi), np.ones(NYi)]

    # Boundary conditions
    dataNYi[1][0] = -1.  # Neumann
    dataNYi[1][-1] = -1.  # Neumann

    # Their positions
    offsets = np.array([-1, 0, 1])
    DXX = sp.dia_matrix((dataNXi, offsets), shape=(NXi, NXi)) * dx_2
    DYY = sp.dia_matrix((dataNYi, offsets), shape=(NYi, NYi)) * dy_2
    # print(DXX.todense())
    # print(DYY.todense())

    # 2D Laplace operator
    LAP = sp.kron(DXX, sp.eye(NYi, NYi)) + sp.kron(sp.eye(NXi, NXi), DYY)

    # BUILD CORRECTION MATRIX

    # # Upper Diagonal terms
    dataNYNXi = [np.zeros(NYi * NXi), np.zeros(NYi * NXi)]
    offset = np.array([-1, 1])

    # Fix coef: 2+(-1) = 1 ==> Dirichlet at a single point
    dataNYNXi[0][0] = -1 * dy_2
    dataNYNXi[1][1] = -1 * dy_2

    LAP0 = sp.dia_matrix((dataNYNXi, offset), shape=(NYi * NXi, NYi * NXi))

    return LAP + LAP0


def BuildLAPMixed():
    """
    Laplacian matrix for Phi with mixed BC (3 Neumann and 1 Dirichlet)

    The value is set at one point (here [0][1]) to ensure uniqueness

    """
    # Dropping ghost points (-2)
    NXi = nx
    NYi = ny

    # 1D Laplace operator

    # X-axis
    # Diagonal terms
    dataNXi = [np.ones(NXi), -2 * np.ones(NXi), np.ones(NXi)]

    # Boundary conditions
    dataNXi[1][0] = -1.  # Neumann
    dataNXi[1][-1] = -1.  # Neumann

    # Y-axis
    # Diagonal terms
    dataNYi = [np.ones(NYi), -2 * np.ones(NYi), np.ones(NYi)]

    # Boundary conditions
    dataNYi[1][0] = -1.  # Neumann
    dataNYi[1][-1] = -1.  # Neumann

    # Their positions
    offsets = np.array([-1, 0, 1])
    DXX = sp.dia_matrix((dataNXi, offsets), shape=(NXi, NXi)) * dx_2
    DYY = sp.dia_matrix((dataNYi, offsets), shape=(NYi, NYi)) * dy_2
    # print(DXX.todense())
    # print(DYY.todense())

    # 2D Laplace operator
    LAP = sp.kron(DXX, sp.eye(NYi, NYi)) + sp.kron(sp.eye(NXi, NXi), DYY)

    # add -2*dx_2 to the diagonal of the last block of the matrix
    # # Upper Diagonal terms
    dataNYNXi = [np.zeros(NYi * NXi)]
    offset = np.array([0])

    # last ny points equal to -2 * dx_2    
    dataNYNXi[0][-NYi:] = -2 * dx_2

    LAP0 = sp.dia_matrix((dataNYNXi, offset), shape=(NYi * NXi, NYi * NXi))

    tmp = LAP + LAP0
    # print(LAP0.todense())

    return LAP + LAP0


def Mine():
    """
    Laplacian matrix for Phi with mixed BC (3 Neumann and 1 Dirichlet)

    The value is set at one point (here [0][1]) to ensure uniqueness

    """
    # Dropping ghost points (-2)
    NXi = nx
    NYi = ny

    # 1D Laplace operator

    # X-axis
    # Diagonal terms
    lamb = dt/(dx*dx)
    mu = dt/(dy*dy)

    print(lamb, mu)

    dataNXi = [-lamb*np.ones(NXi), 2*lamb * np.ones(NXi), -lamb*np.ones(NXi)]

    # Boundary conditions
    # dataNXi[1][0] = -dt/(dx*dx)
    # dataNXi[1][-1] = -dt/(dx*dx)

    # Y-axis
    # Diagonal terms
    dataNYi = [-mu*np.ones(NYi), 2*mu * np.ones(NYi),-mu* np.ones(NYi)]

    # Boundary conditions
    # dataNYi[1][0] = -1.  # Neumann
    dataNYi[1][-1] = mu

    # Their positions
    offsets = np.array([-1, 0, 1])
    DXX = sp.dia_matrix((dataNXi, offsets), shape=(NXi, NXi))
    DYY = sp.dia_matrix((dataNYi, offsets), shape=(NYi, NYi))
    # print(DXX.todense())
    # print(DYY.todense())

    # 2D Laplace operator
    LAP = sp.kron(DXX, sp.eye(NYi, NYi)) + sp.kron(sp.eye(NXi, NXi), DYY)
    ident = sp.eye(NXi*NYi)

    # add -2*dx_2 to the diagonal of the last block of the matrix
    # # Upper Diagonal terms
    # dataNYNXi = [np.zeros(NYi * NXi)]
    # offset = np.array([0])

    # # last ny points equal to -2 * dx_2
    # dataNYNXi[0][-NYi:] = -2 * dx_2

    # LAP0 = sp.dia_matrix((dataNYNXi, offset), shape=(NYi * NXi, NYi * NXi))

    # print(LAP0.todense())

    return LAP + ident

In [2]:
NX = 12
NY = 7
nx = NX - 2
ny = NY - 2
LX = 1.
LY = 1.
dx = LX / (nx)
dy = LY / (ny)
dt = 0.0001
dx_2 = 1. / (dx * dx)
dy_2 = 1. / (dy * dy)
print(dx_2, dy_2)
LAPoisson = BuildLaPoisson()
LANeu = BuildLAPN()
LAMixed = BuildLAPMixed()
LAMine = Mine()

99.99999999999999 24.999999999999996
0.009999999999999998 0.0024999999999999996


In [3]:
np.set_printoptions(threshold=np.inf, linewidth=1800)

In [4]:
LAMine.todense()

matrix([[ 1.025 , -0.0025,  0.    ,  0.    ,  0.    , -0.01  ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
        [-0.0025,  1.025 , -0.0025,  0.    ,  0.    ,  0.    , -0.01  ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
        [ 0.    , -0.0025,  1.025 , -0.0025,  0.    ,  0.    ,  0.    , -0.01  ,

In [5]:
A = -LAMixed.todense()
A.shape
A

matrix([[ 89., -25.,  -0.,  -0.,  -0., -64.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.],
        [-25., 114., -25.,  -0.,  -0.,  -0., -64.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.],
        [ -0., -25., 114., -25.,  -0.,  -0.,  -0., -64.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.],
        [ -0.,  -0., -25., 114., -25.,  -0.,  -0.,  -0., -64.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.],


In [59]:
b = np.zeros(nx*ny)
s = 0.000256
b[ny -1] = -s
b[-1] = s

In [60]:
b

array([ 0.      ,  0.      ,  0.      , -0.000256,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.000256])

In [61]:
np.max(la.eigvals(A))

300.8369345737659

In [49]:
#cholesky decomposition of A (with ones on the diagonal)
L = la.cholesky(-A)
L

matrix([[ 8.94427191,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [-0.        ,  9.79795897,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [-0.        , -1.63299316,  9.66091783,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.      

In [50]:
L.T

matrix([[ 8.94427191, -0.        , -0.        , -0.        , -7.15541753, -0.        , -0.        , -0.        , -0.        , -0.        , -0.        , -0.        , -0.        , -0.        , -0.        , -0.        , -0.        , -0.        , -0.        , -0.        , -0.        , -0.        , -0.        , -0.        , -0.        , -0.        , -0.        , -0.        , -0.        , -0.        , -0.        , -0.        ],
        [ 0.        ,  9.79795897, -1.63299316, -0.        , -0.        , -6.53197265, -0.        , -0.        , -0.        , -0.        , -0.        , -0.        , -0.        , -0.        , -0.        , -0.        , -0.        , -0.        , -0.        , -0.        , -0.        , -0.        , -0.        , -0.        , -0.        , -0.        , -0.        , -0.        , -0.        , -0.        , -0.        , -0.        ],
        [ 0.        ,  0.        ,  9.66091783, -1.65615734, -0.        , -1.10410489, -6.62462937, -0.        , -0.        , -0.        , -0.      

In [64]:
la.norm(A)

927.1720444448268

In [65]:
la.norm(Ainv)

1.557989959249139

In [62]:
Ainv = la.inv(A)

In [63]:
(Ainv@b).reshape(nx,ny)

matrix([[ 4.15118990e-07, -4.15118990e-07, -2.11161055e-06, -6.68758693e-06],
        [ 5.18898737e-07, -9.47758462e-08, -1.39173935e-06, -3.83158102e-06],
        [ 7.76097131e-07,  3.96389528e-07, -3.86148606e-07, -1.58553554e-06],
        [ 1.12822242e-06,  9.88262535e-07,  7.23654338e-07,  3.60663217e-07],
        [ 1.51533769e-06,  1.61129762e-06,  1.85805301e-06,  2.21611419e-06],
        [ 1.87846298e-06,  2.19663384e-06,  2.96462524e-06,  4.16108046e-06],
        [ 2.16204555e-06,  2.66951492e-06,  3.96408152e-06,  6.40516054e-06],
        [ 2.31876077e-06,  2.94562169e-06,  4.67690969e-06,  9.25951037e-06]])

In [35]:
la.solve(A, b).reshape(nx, ny)

array([[ 4.15118990e-07, -4.15118990e-07, -2.11161055e-06, -6.68758693e-06],
       [ 5.18898737e-07, -9.47758462e-08, -1.39173935e-06, -3.83158102e-06],
       [ 7.76097131e-07,  3.96389528e-07, -3.86148606e-07, -1.58553554e-06],
       [ 1.12822242e-06,  9.88262535e-07,  7.23654338e-07,  3.60663217e-07],
       [ 1.51533769e-06,  1.61129762e-06,  1.85805301e-06,  2.21611419e-06],
       [ 1.87846298e-06,  2.19663384e-06,  2.96462524e-06,  4.16108046e-06],
       [ 2.16204555e-06,  2.66951492e-06,  3.96408152e-06,  6.40516054e-06],
       [ 2.31876077e-06,  2.94562169e-06,  4.67690969e-06,  9.25951037e-06]])

In [75]:
lg.eigs(LAMixed, k=10, which='SM')

(array([-0.1070825 +0.j, -0.53529786+0.j, -0.96305467+0.j, -1.39127002+0.j, -1.81811023+0.j, -2.67408239+0.j, -2.67133358+0.j, -3.09954894+0.j, -3.95002642+0.j, -4.38236131+0.j]),
 array([[ 2.94588386e-02+0.j, -4.16387832e-02+0.j, -2.94272933e-02+0.j,  4.15941954e-02+0.j,  4.15718895e-02+0.j,  4.15273733e-02+0.j, -2.93642366e-02+0.j, -4.15050674e-02+0.j, -4.14604795e-02+0.j,  4.14383885e-02+0.j],
        [ 2.94588386e-02+0.j, -4.14604795e-02+0.j, -2.94272933e-02+0.j,  4.14160826e-02+0.j,  4.08605829e-02+0.j,  4.08168284e-02+0.j, -2.93642366e-02+0.j, -4.13273363e-02+0.j, -3.98671766e-02+0.j,  4.07293662e-02+0.j],
        [ 2.94588386e-02+0.j, -4.11046357e-02+0.j, -2.94272933e-02+0.j,  4.10606199e-02+0.j,  3.94501405e-02+0.j,  3.94078963e-02+0.j, -2.93642366e-02+0.j, -4.09726353e-02+0.j, -3.67418004e-02+0.j,  3.93234531e-02+0.j],
        [ 2.94588386e-02+0.j, -4.05727756e-02+0.j, -2.94272933e-02+0.j,  4.05293292e-02+0.j,  3.73646952e-02+0.j,  3.73246841e-02+0.j, -2.93642366e-02+0.j, -4.0

In [79]:
M = np.arange(1, 10).reshape(3,3)+2*np.eye(3)
M = M + M.T
M

array([[ 6.,  6., 10.],
       [ 6., 14., 14.],
       [10., 14., 22.]])

In [80]:
la.eig(M)

EigResult(eigenvalues=array([36.91647287,  1.08352713,  4.        ]), eigenvectors=array([[-0.35162514, -0.84243284,  0.40824829],
       [-0.55335618, -0.1647127 , -0.81649658],
       [-0.75508721,  0.51300744,  0.40824829]]))

In [81]:
b = np.arange(1, 4)
b

array([1, 2, 3])

In [82]:
la.solve(M, b)

array([-2.50000000e-01,  1.09039761e-16,  2.50000000e-01])